In [52]:
import pandas as pd

In [53]:
file_path = 'train.csv'
data = pd.read_csv(file_path)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [55]:
missing_values = data.isnull().sum()

In [56]:
data_cleaned = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [57]:
data_cleaned['Age'].fillna(data_cleaned['Age'].median(), inplace=True)
data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0], inplace=True)

/tmp/ipykernel_4382/4197230416.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_cleaned['Age'].fillna(data_cleaned['Age'].median(), inplace=True)
/tmp/ipykernel_4382/4197230416.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

In [58]:
label_encoder = LabelEncoder()
data_cleaned['Sex'] = label_encoder.fit_transform(data_cleaned['Sex'])
data_cleaned['Embarked'] = label_encoder.fit_transform(data_cleaned['Embarked'])

In [59]:
X = data_cleaned.drop('Survived', axis=1)
y = data_cleaned['Survived']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
data_cleaned.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [63]:
rf = RandomForestClassifier(random_state=42)
dt = DecisionTreeClassifier(random_state=42)
nb = GaussianNB()

In [64]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

In [65]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_grid, n_iter=10, cv=3, verbose=1, random_state=42)
dt_random = RandomizedSearchCV(estimator=dt, param_distributions=dt_param_grid, n_iter=10, cv=3, verbose=1, random_state=42)

In [66]:
rf_random.fit(X_train, y_train)
dt_random.fit(X_train, y_train)
nb.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits


GaussianNB()

In [67]:
rf_pred = rf_random.predict(X_test)
dt_pred = dt_random.predict(X_test)
nb_pred = nb.predict(X_test)

In [68]:
rf_acc = accuracy_score(y_test, rf_pred)
dt_acc = accuracy_score(y_test, dt_pred)
nb_acc = accuracy_score(y_test, nb_pred)

In [69]:
dt_precision = precision_score(y_test, dt_pred)
rf_precision = precision_score(y_test, rf_pred)
nb_precision = precision_score(y_test, nb_pred)

In [70]:
dt_recall = recall_score(y_test, dt_pred)
rf_recall = recall_score(y_test, rf_pred)
nb_recall = recall_score(y_test, nb_pred)

In [75]:
print("Decision Tree Precision:", dt_precision)
print("RandomForest Precision:", rf_precision)
print("Naive Bayes Precision:", nb_precision)

print("Decision Tree Recall:", dt_recall)
print("RandomForest Recall:", rf_recall)
print("Naive Bayes Recall:", nb_recall)

print("Decision Tree Accuracy:", dt_acc)
print("RandomForest Accuracy:", rf_acc)
print("Naive Bayes Accyracy:", nb_acc)

Decision Tree Precision: 0.8548387096774194
RandomForest Precision: 0.859375
Naive Bayes Precision: 0.7125
Decision Tree Recall: 0.7162162162162162
RandomForest Recall: 0.7432432432432432
Naive Bayes Recall: 0.7702702702702703
Decision Tree Accuracy: 0.8324022346368715
RandomForest Accuracy: 0.8435754189944135
Naive Bayes Accyracy: 0.776536312849162
